### Shared Word File Process

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # always must be False in this part
native_word = True # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.
print(f"{file_ext}")

In [ ]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

### Twogram In Threegram

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_shared_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx")
#df_shared_file = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx")
df_shared_file

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [ ]:
def word_in_wordgroup(df, list_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            #word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(10)  # Option
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(100) 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{list_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [ ]:
#df_shared_count = word_count_result(df_shared_file,["threegram"])
#df_shared_count

In [ ]:
df_shared_file["twogram"].nunique()

In [ ]:
df_two_in_three = word_in_wordgroup(df_shared_file, "twogram", "threegram")
df_two_in_three

In [ ]:
df_two_in_three["twogram"].nunique()

In [ ]:
df_shared_select_twogram = df_shared_file.loc[:,["twogram","freq_twogram"]]
df_shared_select_twogram

In [ ]:
set_shared_twogram = set(df_shared_select_twogram["twogram"])
set_two_three = set(df_two_in_three["twogram"])

In [ ]:
df_twogram_in_threegram = pd.DataFrame(set_two_three, columns=["twogram"])  # columns=["twogram_in_threegram"]
df_twogram_in_threegram

In [ ]:
df_twogram_in_threegram_freq = pd.merge(df_twogram_in_threegram, df_shared_select_twogram, how="left", on="twogram")
df_twogram_in_threegram_freq.sort_values(by="freq_twogram", ascending=False, inplace=True)
df_twogram_in_threegram_freq.rename(columns={"twogram":"twogram_in_threegram","freq_twogram":"freq_two_in_three"}, inplace=True)
df_twogram_in_threegram_freq.reset_index(drop=True, inplace=True)
df_twogram_in_threegram_freq

In [ ]:
df_twogram_diff = pd.DataFrame(set_shared_twogram.difference(set_two_three), columns=["twogram"])
df_twogram_diff

In [ ]:
df_twogram_diff_freq = pd.merge(df_twogram_diff, df_shared_select_twogram, how="left", on="twogram")
df_twogram_diff_freq.sort_values(by="freq_twogram", ascending=False, inplace=True)
df_twogram_diff_freq.reset_index(drop=True, inplace=True)
df_twogram_diff_freq

In [ ]:
df_shared_file["twogram"] = df_twogram_diff_freq["twogram"]
df_shared_file["freq_twogram"] = df_twogram_diff_freq["freq_twogram"]

In [ ]:
df_shared_twogram_process = pd.concat([df_shared_file,df_twogram_in_threegram_freq], axis=1)
df_shared_twogram_process

In [ ]:
df_shared_twogram_process.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}2.xlsx", index=False)

##### Concat Result With Comma

In [ ]:
df_word_order_twogram = word_in_wordgroup(df_shared_twogram_process, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_twogram_process, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_twogram_process, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_twogram_process, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_twogram_process, "word", "sentence")
df_word_order_twogram_threegram = word_in_wordgroup(df_shared_twogram_process, "word", "twogram_in_threegram") 

In [ ]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()
df_word_order_join_twogram_threegram = df_word_order_twogram_threegram.groupby(["word"])["twogram_in_threegram"].apply(", ".join).reset_index()

In [ ]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence,df_word_order_join_twogram_threegram]

In [ ]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

In [ ]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence","twogram_in_threegram"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

In [ ]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}2.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}2.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

### Prefix Suffix Result Analysis

In [1]:
import os
import multiprocessing
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# word sample
word_sample = True  # True, False
word_sample_num = 20

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # always must be True in this part
native_word = False # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.
print(f"{file_ext}")

4


In [3]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            if word_sample:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)  # Option
            else:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [4]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [5]:
df_word_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
df_word_prefix_suffix = df_word_prefix_suffix[["search_word","word"]]
df_word_prefix_suffix

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
162,şey,şeyler
163,şey,şeylerden
164,şey,şeylere
165,şey,şeyleri


In [6]:
df_ety_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
df_ety_prefix_suffix = df_ety_prefix_suffix[["search_word","word"]]
df_ety_prefix_suffix

,search_word,word
0,abluka,abluka
1,abluka,ablukaya
2,abluka,ablukayı
3,absorbe,absorbe
4,absürt,absürt
...,...,...
4896,şut,şutunu
4897,şık,şık
4898,şırınga,şırınga
4899,şırınga,şırıngayla


In [7]:
if file_ext == "3":
    df_all_word = pd.concat([df_word_prefix_suffix,df_ety_prefix_suffix],axis=0)
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "4":
    df_all_word = df_ety_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "5":
    df_all_word = df_word_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
else:
    pass
    
df_all_word

,search_word,word
0,abluka,abluka
1,abluka,ablukaya
2,abluka,ablukayı
3,absorbe,absorbe
4,absürt,absürt
...,...,...
4896,şut,şutunu
4897,şık,şık
4898,şırınga,şırınga
4899,şırınga,şırıngayla


In [8]:
df_all_word.search_word.nunique()

1548

In [9]:
df_all_word.word.nunique()

4795

In [10]:
df_shared_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx")
#df_shared_all = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx")
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,biraz,1269641.0,milyon dolar,6377,milyon dolardan fazla,279.0,fazla kaliteli normal banka,20.0,radyo televizyon telefon bar video,10.0,telefon numarası,232.0
1,fazla,692043.0,biraz fazla,5111,standart kablolu orkestrası,191.0,sistemleri telekom sektöründe lider,17.0,modülü reaksiyon kontrol sistemi valfleri,10.0,standart prosedür,192.0
2,kontrol,272328.0,telefon numarası,1473,kongre onur madalyası,113.0,projenin rutin kopya mikrofilm,17.0,NaN,NaN,alo santral,149.0
3,doktor,266714.0,biraz kahve,1354,milyon amerikan doları,84.0,amerikan müzisyenler sendika federasyonu,16.0,NaN,NaN,genel alarm,139.0
4,bazı,249261.0,milyon dolarlık,1102,adres telefon numarası,48.0,alfa alfa bravo eko,13.0,NaN,NaN,biraz fazla,138.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7535,NaN,NaN,sisteminde mesajlar,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7536,NaN,NaN,ekonomik potansiyelini,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7537,NaN,NaN,sistemleri normal,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7538,NaN,NaN,ekranı biraz,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_all, "word", "sentence") 

In [12]:
df_word_order_twogram = pd.merge(df_word_order_twogram,df_all_word, how="inner", on="word")
df_word_order_threegram = pd.merge(df_word_order_threegram,df_all_word, how="inner", on="word")
df_word_order_fourgram = pd.merge(df_word_order_fourgram,df_all_word, how="inner", on="word")
df_word_order_fivegram = pd.merge(df_word_order_fivegram,df_all_word, how="inner", on="word")
df_word_order_sentence = pd.merge(df_word_order_sentence,df_all_word, how="inner", on="word")

In [13]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["search_word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["search_word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["search_word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["search_word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["search_word"])["sentence"].apply(", ".join).reset_index()

In [14]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [15]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['search_word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all.rename(columns={"search_word":"word"}, inplace=True)
df_word_order_join_all

,word,twogram,threegram,fourgram,fivegram,sentence
0,abluka,abluka riskini,NaN,NaN,NaN,NaN
1,absorbe,radyasyonu absorbe,NaN,NaN,NaN,NaN
2,acente,acente kopyası,NaN,NaN,NaN,NaN
3,adaptasyon,atmosfere adaptasyon,NaN,NaN,NaN,NaN
4,adapte,"strese adapte, okula adapte",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1124,paparazzi,NaN,NaN,NaN,NaN,basına paparazzilere
1125,sedan,NaN,NaN,NaN,NaN,gri sedan
1126,sübjektif,NaN,NaN,NaN,NaN,biraz sübjektif
1127,termometre,NaN,NaN,NaN,NaN,diferansiyel termometre


In [16]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="left", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,kontrol,272328.0,"numarayı kontrol, kontrol paneli, listeyi kont...","modülü reaksiyon kontrol, reaksiyon kontrol si...","reaksiyon kontrol sistemi valfleri, modülü rea...",modülü reaksiyon kontrol sistemi valfleri,"bilet kontrol, rutin kontrol, mikrofon kontrol..."
1,doktor,266714.0,"doktor fane, şanslar doktor, doktor raporu, do...",doktor bazı testler,"şampiyonu doktor üniversite profesörü, olimpiy...",NaN,"doktor fane, pardon doktor, doktor park, bravo..."
2,polis,247969.0,"polis şefi, fazla polis, polis raporu, alo pol...","polis robot polis, metroda polis operasyon, am...","polis istasyonunda alarm otomatik, bazı polisl...",NaN,"alo polis, polis şefi, süper polis, polis akad..."
3,dolar,199343.0,"milyon dolar, milyonlarca dolar, dolar avans, ...","dolar biraz fazla, milyon dolar fazla, milyon ...",NaN,NaN,"milyonlarca dolar, milyon dolar, amerikan doları"
4,vay,162887.0,"vay termostat, vay polis, vay süper, vay enerj...",NaN,NaN,NaN,"vay süper, vay çakal, vay doktor"
...,...,...,...,...,...,...,...
1124,barmen,NaN,"barmenlik kursu, barmenlik sertifikası, barmen...",NaN,NaN,NaN,NaN
1125,ekolayzer,NaN,ekolayzerin modern,NaN,NaN,NaN,NaN
1126,metres,NaN,"metresinin metresi, fransız metresi, metresi m...",NaN,NaN,NaN,robotun metresi
1127,teras,NaN,"terasta latin, terastan terasa, terası kontrol...",NaN,NaN,NaN,NaN


In [17]:
df_word_order_join_all.word.nunique()

1129

In [18]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}3.xlsx", index=False)

#### Copy Move And Delete

In [19]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}3.xlsx")
output_file2

['Turkish_Dutch_Shared_Join_Result_Without_Frequency43.xlsx']

In [20]:
for l in output_file2:
    source = l # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [21]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

### Prefix Suffix Shared File Word Result Select

In [22]:
import os
import multiprocessing
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [23]:
## language pair (same previous part parameter)
#lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# word sample
word_sample_num = 20

print(f"{file_ext}")

4


In [24]:
def lower_strip_func(x):
    try:
        var_low = x.lower()
        var_out = var_low.strip()
    except:
        var_out = x
    return var_out 

In [25]:
df_shared_process_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}3.xlsx")
df_shared_process_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,kontrol,272328.0,"numarayı kontrol, kontrol paneli, listeyi kont...","modülü reaksiyon kontrol, reaksiyon kontrol si...","reaksiyon kontrol sistemi valfleri, modülü rea...",modülü reaksiyon kontrol sistemi valfleri,"bilet kontrol, rutin kontrol, mikrofon kontrol..."
1,doktor,266714.0,"doktor fane, şanslar doktor, doktor raporu, do...",doktor bazı testler,"şampiyonu doktor üniversite profesörü, olimpiy...",NaN,"doktor fane, pardon doktor, doktor park, bravo..."
2,polis,247969.0,"polis şefi, fazla polis, polis raporu, alo pol...","polis robot polis, metroda polis operasyon, am...","polis istasyonunda alarm otomatik, bazı polisl...",NaN,"alo polis, polis şefi, süper polis, polis akad..."
3,dolar,199343.0,"milyon dolar, milyonlarca dolar, dolar avans, ...","dolar biraz fazla, milyon dolar fazla, milyon ...",NaN,NaN,"milyonlarca dolar, milyon dolar, amerikan doları"
4,vay,162887.0,"vay termostat, vay polis, vay süper, vay enerj...",NaN,NaN,NaN,"vay süper, vay çakal, vay doktor"
...,...,...,...,...,...,...,...
1124,barmen,NaN,"barmenlik kursu, barmenlik sertifikası, barmen...",NaN,NaN,NaN,NaN
1125,ekolayzer,NaN,ekolayzerin modern,NaN,NaN,NaN,NaN
1126,metres,NaN,"metresinin metresi, fransız metresi, metresi m...",NaN,NaN,NaN,robotun metresi
1127,teras,NaN,"terasta latin, terastan terasa, terası kontrol...",NaN,NaN,NaN,NaN


In [26]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all = df_word_all.loc[:,["word","frequency"]]
df_word_all["word"] = df_word_all["word"].apply(lambda x: lower_strip_func(x))
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [27]:
df_twogram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_all = df_twogram_all.loc[:,["twogram","frequency"]]
df_twogram_all["twogram"] = df_twogram_all["twogram"].apply(lambda x: lower_strip_func(x))
df_twogram_all

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [28]:
df_threegram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")
df_threegram_all = df_threegram_all.loc[:,["threegram","frequency"]]
df_threegram_all["threegram"] = df_threegram_all["threegram"].apply(lambda x: lower_strip_func(x))
df_threegram_all

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [29]:
df_fourgram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")
df_fourgram_all = df_fourgram_all.loc[:,["fourgram","frequency"]]
df_fourgram_all["fourgram"] = df_fourgram_all["fourgram"].apply(lambda x: lower_strip_func(x))
df_fourgram_all

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
3052209,fikret miloya merhaba de,5
3052210,fikret millsin oğlu mu,5
3052211,fikret millet iyi iş,5
3052212,fikret millet bize bakıyor,5


In [30]:
df_fivegram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")
df_fivegram_all = df_fivegram_all.loc[:,["fivegram","frequency"]]
df_fivegram_all["fivegram"] = df_fivegram_all["fivegram"].apply(lambda x: lower_strip_func(x))
df_fivegram_all

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [31]:
df_sentence_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/Sentence/Merge/Sentence_Merge.csv")
df_sentence_all = df_sentence_all.loc[:,["sentence","frequency"]]
df_sentence_all["sentence"] = df_sentence_all["sentence"].apply(lambda x: lower_strip_func(x))
df_sentence_all

,sentence,frequency
0,evet,1948596
1,fikret,1533918
2,hayır,1250401
3,tamam,882921
4,ne,753105
...,...,...
2913965,hayır ben bir pikap kamyon alacağım,6
2913966,eminim uydurmuştur,6
2913967,hemen bir kement ile onu yakalar ve aşağı çekerim,6
2913968,tabii gerçek bir profesyonel o,6


In [32]:
for i in range(len(df_shared_process_all)):
    # column result
    try:
        # column result
        df_two_var = pd.DataFrame(df_shared_process_all.loc[i,"twogram"].split(", "), columns=["twogram"])
        # merge with all
        df_two_var_merge = pd.merge(df_two_var, df_twogram_all, how="left", on="twogram")
        df_two_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_two_var_merge_select = df_two_var_merge.head(word_sample_num)
        # to list value
        df_two_var_list = df_two_var_merge_select["twogram"].to_list()
        # list join
        df_two_var_list_join = ", ".join(df_two_var_list)
        # change value
        df_shared_process_all.loc[i,"twogram"] = df_two_var_list_join
    except:
        pass
    try:
        # column result
        df_three_var = pd.DataFrame(df_shared_process_all.loc[i,"threegram"].split(", "), columns=["threegram"])
        # merge with all
        df_three_var_merge = pd.merge(df_three_var, df_threegram_all, how="left", on="threegram")
        df_three_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_three_var_merge_select = df_three_var_merge.head(word_sample_num)
        # to list value
        df_three_var_list = df_three_var_merge_select["threegram"].to_list()
        # list join
        df_three_var_list_join = ", ".join(df_three_var_list)
        # change value
        df_shared_process_all.loc[i,"threegram"] = df_three_var_list_join
    except:
        pass
    try:
        # column result
        df_four_var = pd.DataFrame(df_shared_process_all.loc[i,"fourgram"].split(", "), columns=["fourgram"])
        # merge with all
        df_four_var_merge = pd.merge(df_four_var, df_fourgram_all, how="left", on="fourgram")
        df_four_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_four_var_merge_select = df_four_var_merge.head(word_sample_num)
        # to list value
        df_four_var_list = df_four_var_merge_select["fourgram"].to_list()
        # list join
        df_four_var_list_join = ", ".join(df_four_var_list)
        # change value
        df_shared_process_all.loc[i,"fourgram"] = df_four_var_list_join
    except:
        pass
    try:
        # column result
        df_five_var = pd.DataFrame(df_shared_process_all.loc[i,"fivegram"].split(", "), columns=["fivegram"])
        # merge with all
        df_five_var_merge = pd.merge(df_five_var, df_fivegram_all, how="left", on="fivegram")
        df_five_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_five_var_merge_select = df_five_var_merge.head(word_sample_num)
        # to list value
        df_five_var_list = df_five_var_merge_select["fivegram"].to_list()
        # list join
        df_five_var_list_join = ", ".join(df_five_var_list)
        # change value
        df_shared_process_all.loc[i,"fivegram"] = df_five_var_list_join
    except:
        pass
    try:
        # column result
        df_sentence_var = pd.DataFrame(df_shared_process_all.loc[i,"sentence"].split(", "), columns=["sentence"])
        # merge with all
        df_sentence_var_merge = pd.merge(df_sentence_var, df_sentence_all, how="left", on="sentence")
        df_sentence_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_sentence_var_merge_select = df_sentence_var_merge.head(word_sample_num)
        # to list value
        df_sentence_var_list = df_sentence_var_merge_select["sentence"].to_list()
        # list join
        df_sentence_var_list_join = ", ".join(df_sentence_var_list)
        # change value
        df_shared_process_all.loc[i,"sentence"] = df_sentence_var_list_join
    except:
        pass      

df_shared_process_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,kontrol,272328.0,"numarayı kontrol, biraz kontrolden, kontrol pa...","modülü reaksiyon kontrol, reaksiyon kontrol si...","reaksiyon kontrol sistemi valfleri, modülü rea...",modülü reaksiyon kontrol sistemi valfleri,"bilet kontrol, sistem kontrolü, rutin kontrol,..."
1,doktor,266714.0,"doktor fane, şanslar doktor, doktor raporu, do...",doktor bazı testler,"şampiyonu doktor üniversite profesörü, olimpiy...",NaN,"doktor fane, pardon doktor, doktor park, bravo..."
2,polis,247969.0,"polis şefi, fazla polis, polis raporu, alo pol...","düzine fransız polisi, polis robot polis, film...","polis istasyonunda alarm otomatik, bazı polisl...",NaN,"alo polis, polis şefi, metro polisi, süper pol..."
3,dolar,199343.0,"milyon dolar, milyon dolarlık, dolardan fazla,...","milyon dolardan fazla, milyon amerikan doları,...",NaN,NaN,"milyonlarca dolar, amerikan doları, milyon dolar"
4,vay,162887.0,"vay termostat, vay polis, vay süper, vay enerj...",NaN,NaN,NaN,"vay süper, vay çakal, vay doktor"
...,...,...,...,...,...,...,...
1124,barmen,NaN,"barmenlik kursu, otelin barmeni, barmenlik ser...",NaN,NaN,NaN,NaN
1125,ekolayzer,NaN,ekolayzerin modern,NaN,NaN,NaN,NaN
1126,metres,NaN,"metresinin metresi, metresinin metresi, fransı...",NaN,NaN,NaN,robotun metresi
1127,teras,NaN,"terası kontrol, terasta latin, terastan terasa...",NaN,NaN,NaN,NaN


In [33]:
#df_shared_process_all.sort_values(by="frequency", ascending=False, inplace=True)

In [34]:
df_shared_process_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Select_Result_Without_Frequency{file_ext}4.xlsx", index=False)

#### Copy Move And Delete

In [35]:
output_file3 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*_Select_*{file_ext}4.xlsx")
output_file3

['Turkish_Dutch_Shared_Join_Select_Result_Without_Frequency44.xlsx']

In [36]:
for l in output_file3:
    source = l # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [37]:
for j in output_file3:
    try:
        os.remove(j)
    except:
        pass

### Word Count Analysis

In [ ]:
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [ ]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/Deploy Result Manuel").mkdir(parents=True, exist_ok=True)

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

#### Hybrid Words

In [ ]:
df_hybrid = pd.read_excel("Turkish English manual selected 2 gram hybrids 3.xlsx", sheet_name="2 gram hybrid")
df_hybrid

In [ ]:
df_hybrid_count = word_count_result(df_hybrid, ["twogram_pair1","twogram_pair2","twogram_pair3","twogram_pair4"])
df_hybrid_count

In [ ]:
df_hybrid_count_merge = pd.merge(df_hybrid,df_hybrid_count,how="left",on="word")
df_hybrid_count_merge

In [ ]:
df_hybrid_count_merge2 = pd.merge(df_hybrid,df_hybrid_count,how="outer",on="word")
df_hybrid_count_merge2

In [ ]:
writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_Hybrid_Word_Count.xlsx", engine='xlsxwriter')

In [ ]:
df_hybrid_count_merge.to_excel(writer, sheet_name='28_Hybrid_Word_Count', index=False)
df_hybrid_count_merge2.to_excel(writer, sheet_name='All_Hybrid_Word_Count', index=False)

In [ ]:
writer.save()

#### Target Words

In [ ]:
df_target = pd.read_excel("Turkish English manual selected 2 gram hybrids 3.xlsx", sheet_name="2 gram target")
df_target

In [ ]:
df_target_count = word_count_result(df_target, ["twogram_1","twogram_2","twogram_3","twogram_4"])
df_target_count

In [ ]:
df_target_count_merge = pd.merge(df_target,df_target_count,how="left",on="word")
df_target_count_merge

In [ ]:
df_target_count_merge2 = pd.merge(df_target,df_target_count,how="outer",on="word")
df_target_count_merge2

In [ ]:
writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_Target_Word_Count.xlsx", engine='xlsxwriter')

In [ ]:
df_target_count_merge.to_excel(writer2, sheet_name='28_Target_Word_Count', index=False)
df_target_count_merge2.to_excel(writer2, sheet_name='All_Target_Word_Count', index=False)

In [ ]:
writer2.save()

#### All Target Hybrid Word Count 

In [ ]:
df_target_count

In [ ]:
df_hybrid_count

In [ ]:
df_all_word = pd.concat([df_target_count, df_hybrid_count], axis=0)
df_all_word

In [ ]:
df_all_word.groupby("word")[["word_count"]].sum().reset_index(inplace=True)
df_all_word.sort_values(by="word_count", ascending=False, inplace=True)
df_all_word

In [ ]:
df_all_word.to_excel(f"{lang_folder}_{lang_pair}_Target_Hybrid_Word_Count.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file4 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*_Word_Count.xlsx")
output_file4

In [ ]:
for o in output_file4:
    source = o # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [ ]:
for p in output_file4:
    try:
        os.remove(p)
    except:
        pass